<a href="https://colab.research.google.com/github/mahesh-keswani/ML-DL-Basics/blob/main/keras_examples/SubClassingInKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist 

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
# y_train are sparse labels and not one-hot encoded
y_train[:5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [4]:
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

In [5]:
# Now the common sequence is: CNN -> BatchNorm -> ReLu and we use this multiple times, so instead we will just create a block and use it multiple times

class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size = 3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(filters=out_channels, kernel_size=kernel_size, padding='same')
        self.bn = layers.BatchNormalization()
    
    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        x = self.bn(x, training = training)
        return tf.nn.relu(x)


In [6]:
model = keras.Sequential(
    [
     CNNBlock(32),
     CNNBlock(64),
     CNNBlock(128),
     layers.Flatten(),
     layers.Dense(10)    
    ]
)

In [7]:
model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits = True), metrics=['accuracy'])

In [8]:
model.fit( x_train, y_train, epochs=5, verbose=2, batch_size=32 )

Epoch 1/5
1875/1875 - 14s - loss: 0.4845 - accuracy: 0.9533
Epoch 2/5
1875/1875 - 11s - loss: 0.0488 - accuracy: 0.9858
Epoch 3/5
1875/1875 - 11s - loss: 0.0430 - accuracy: 0.9865
Epoch 4/5
1875/1875 - 11s - loss: 0.0344 - accuracy: 0.9889
Epoch 5/5
1875/1875 - 11s - loss: 0.0249 - accuracy: 0.9920


In [9]:
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

313/313 - 1s - loss: 0.0445 - accuracy: 0.9873


[0.044517017900943756, 0.9872999787330627]

In [10]:
# Now by creating our own block, we can do anything in call(), like do any pythonic things ifelse, loops or do some debugging stuff
# like printing output shapes of every layer.

class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size = 3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(filters=out_channels, kernel_size=kernel_size, padding='same')
        self.bn = layers.BatchNormalization()
    
    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        x = self.bn(x, training = training)
        print(x.shape)
        return tf.nn.relu(x)


In [11]:
model = keras.Sequential(
    [
     CNNBlock(32),
     CNNBlock(64),
     CNNBlock(128),
     layers.Flatten(),
     layers.Dense(10)    
    ]
)

model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits = True), metrics=['accuracy'])
model.fit( x_train, y_train, epochs=1, verbose=2, batch_size=32 )

(32, 28, 28, 32)
(32, 28, 28, 64)
(32, 28, 28, 128)
(32, 28, 28, 32)
(32, 28, 28, 64)
(32, 28, 28, 128)
(32, 28, 28, 32)
(32, 28, 28, 64)
(32, 28, 28, 128)
1875/1875 - 11s - loss: 0.5225 - accuracy: 0.9518
